In [1263]:
import re
import json
import stanza

import numpy as np
import pandas as pd

from itertools import chain
from ast import literal_eval

from preprocessing_functions import load_txt_as_lst, split_txt, \
run_stanza, flatten_list, segment_series, preprocess_series, write_file

# Lucretius en1893 no section markers

In [2]:
raw_str_path = "/home/craig.car/repos/chiron/align_texts_project/data/lucretius/lucretius_en1893_str.txt"
en1893_raw_lst = load_txt_as_lst(raw_str_path)

In [3]:
en1893_raw_lst[:10]

['REMARKS\n',
 '\n',
 'ON THE\n',
 '   LIFE AND POEM OF LUCRETIUS.\n',
 '  OF the life of Lucretius but little information has reached us.  Ad nos vix tenuis famæ perlabitur aura.\n',
 '\n',
 'That he was a Roman by birth, is inferred from the pas  sages in his poem in which he speaks of the Roman world as  his country,1 and of the Roman language as his native tongue.2\n',
 '\n',
 'As to the time of his birth, it is stated by Eusebius in his  Chronicon, that he was born in the second year of the hundred  and seventy-first Olympiad, or ninety-five years before Christ.  At this period, Ennius had been dead about seventy years ;  Cicero was in his twelfth year; twenty-five years were to  elapse before the birth of Virgil, and four before that of Julius  Caesar. His style, indeed, would make him seem older, but  its antiquated character may be partly affected, in imitation,  perhaps, of Ennius, for whom he expresses great veneration.3\n',
 '\n']

In [4]:
en1893_str = " ".join(en1893_raw_lst)

In [5]:
# load stanza model for lang
lang_ = "en"
stanza_model_ = stanza.Pipeline(lang=lang_, processors='tokenize', use_gpu=True)

2023-07-02 13:29:28 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-07-02 13:29:28 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |

2023-07-02 13:29:28 WARNING: GPU requested, but is not available!
2023-07-02 13:29:28 INFO: Using device: cpu
2023-07-02 13:29:28 INFO: Loading: tokenize
2023-07-02 13:29:28 INFO: Done loading processors!


In [6]:
en1983_sents = preprocess_series(en1893_str, "en", stanza_model_)

segmented str into sentences


In [7]:
len(en1983_sents)

13966

In [8]:
en1983_sents[:10]

['REMARKS',
 'ON THE',
 'LIFE AND POEM OF LUCRETIUS.',
 'OF the life of Lucretius but little information has reached us.',
 'Ad nos vix tenuis famæ perlabitur aura.',
 'That he was a Roman by birth, is inferred from the pas  sages in his poem in which he speaks of the Roman world as  his country,1 and of the Roman language as his native tongue.',
 '2',
 'As to the time of his birth, it is stated by Eusebius in his  Chronicon, that he was born in the second year of the hundred  and seventy-first Olympiad, or ninety-five years before Christ.',
 'At this period, Ennius had been dead about seventy years ;',
 'Cicero was in his twelfth year;']

In [9]:
# path_out = "/home/craig.car/repos/chiron/align_texts_project/data/lucretius/lucretius_en1893_sents.txt"
# write_file(en1983_sents, path_out)

# Lucretius en1893 with section markers

In [10]:
raw_path_markers = "/home/craig.car/repos/chiron/align_texts_project/data/lucretius/lucretius_en1893_str_withsections.txt"
en1893_raw_lst_markers = load_txt_as_lst(raw_path_markers)

In [11]:
en1893_raw_lst_markers[:10]

['#@$% chapter= #@$%\n',
 '\n',
 '#@$%title#@$%REMARKS\n',
 '\n',
 '#@$%title#@$%ON THE\n',
 '   #@$%title#@$%LIFE AND POEM OF LUCRETIUS.\n',
 '  OF the life of Lucretius but little information has reached us.  Ad nos vix tenuis famæ perlabitur aura.\n',
 '\n',
 'That he was a Roman by birth, is inferred from the pas  sages in his poem in which he speaks of the Roman world as  his country,1 and of the Roman language as his native tongue.2\n',
 '\n']

In [12]:
en1893_str_markers = " ".join(en1893_raw_lst_markers)

In [13]:
en1893_str_markers[:1000]

'#@$% chapter= #@$%\n \n #@$%title#@$%REMARKS\n \n #@$%title#@$%ON THE\n    #@$%title#@$%LIFE AND POEM OF LUCRETIUS.\n   OF the life of Lucretius but little information has reached us.  Ad nos vix tenuis famæ perlabitur aura.\n \n That he was a Roman by birth, is inferred from the pas  sages in his poem in which he speaks of the Roman world as  his country,1 and of the Roman language as his native tongue.2\n \n As to the time of his birth, it is stated by Eusebius in his  Chronicon, that he was born in the second year of the hundred  and seventy-first Olympiad, or ninety-five years before Christ.  At this period, Ennius had been dead about seventy years ;  Cicero was in his twelfth year; twenty-five years were to  elapse before the birth of Virgil, and four before that of Julius  Caesar. His style, indeed, would make him seem older, but  its antiquated character may be partly affected, in imitation,  perhaps, of Ennius, for whom he expresses great veneration.3\n \n Concerning his famil

In [14]:
en1893_str_markers = en1893_str_markers.replace("\n", "")

In [15]:
en1893_str_markers[:1000]

'#@$% chapter= #@$%  #@$%title#@$%REMARKS  #@$%title#@$%ON THE    #@$%title#@$%LIFE AND POEM OF LUCRETIUS.   OF the life of Lucretius but little information has reached us.  Ad nos vix tenuis famæ perlabitur aura.  That he was a Roman by birth, is inferred from the pas  sages in his poem in which he speaks of the Roman world as  his country,1 and of the Roman language as his native tongue.2  As to the time of his birth, it is stated by Eusebius in his  Chronicon, that he was born in the second year of the hundred  and seventy-first Olympiad, or ninety-five years before Christ.  At this period, Ennius had been dead about seventy years ;  Cicero was in his twelfth year; twenty-five years were to  elapse before the birth of Virgil, and four before that of Julius  Caesar. His style, indeed, would make him seem older, but  its antiquated character may be partly affected, in imitation,  perhaps, of Ennius, for whom he expresses great veneration.3  Concerning his family nothing is known. The 

## Split by section (including chapters)
Note: "#@$%" added at beginning and end of section markers when extracting text from xml

In [16]:
en1893_str_markers_split = en1893_str_markers.split("#@$%")

In [17]:
en1893_str_markers_split[:20]

['',
 ' chapter= ',
 '  ',
 'title',
 'REMARKS  ',
 'title',
 'ON THE    ',
 'title',
 'LIFE AND POEM OF LUCRETIUS.   OF the life of Lucretius but little information has reached us.  Ad nos vix tenuis famæ perlabitur aura.  That he was a Roman by birth, is inferred from the pas  sages in his poem in which he speaks of the Roman world as  his country,1 and of the Roman language as his native tongue.2  As to the time of his birth, it is stated by Eusebius in his  Chronicon, that he was born in the second year of the hundred  and seventy-first Olympiad, or ninety-five years before Christ.  At this period, Ennius had been dead about seventy years ;  Cicero was in his twelfth year; twenty-five years were to  elapse before the birth of Virgil, and four before that of Julius  Caesar. His style, indeed, would make him seem older, but  its antiquated character may be partly affected, in imitation,  perhaps, of Ennius, for whom he expresses great veneration.3  Concerning his family nothing is kn

In [18]:
# define patterns
chapter_marker = re.compile(" chapter= ") # 4: commentary, prose translation, metric translation, index
title_marker = re.compile("title")
title_commentary = re.compile("ARGUMENT") # startswith
title_book = re.compile("BOOK")
title_index = re.compile("INDEX.")
para_end = re.compile("paragraph end")
note_start = re.compile("note")
note_end = re.compile("note_end")

In [19]:
test = "ARGUMENT. asoidau"

In [20]:
test.startswith("ARGUMENT.")

True

In [21]:
len(en1893_str_markers_split)

5765

In [22]:
chapteridx2chaptername = {}
chapteridx2chaptername[0] = "foreword"
chapteridx2chaptername[1] = "prose_translation"
chapteridx2chaptername[2] = "metric_translation"
chapteridx2chaptername[3] = "index"

In [23]:
en1893_str_markers_split[:25]

['',
 ' chapter= ',
 '  ',
 'title',
 'REMARKS  ',
 'title',
 'ON THE    ',
 'title',
 'LIFE AND POEM OF LUCRETIUS.   OF the life of Lucretius but little information has reached us.  Ad nos vix tenuis famæ perlabitur aura.  That he was a Roman by birth, is inferred from the pas  sages in his poem in which he speaks of the Roman world as  his country,1 and of the Roman language as his native tongue.2  As to the time of his birth, it is stated by Eusebius in his  Chronicon, that he was born in the second year of the hundred  and seventy-first Olympiad, or ninety-five years before Christ.  At this period, Ennius had been dead about seventy years ;  Cicero was in his twelfth year; twenty-five years were to  elapse before the birth of Virgil, and four before that of Julius  Caesar. His style, indeed, would make him seem older, but  its antiquated character may be partly affected, in imitation,  perhaps, of Ennius, for whom he expresses great veneration.3  Concerning his family nothing is kn

## remove empty strings

In [24]:
df = pd.DataFrame(en1893_str_markers_split)
df.head()

,0
0,
1,chapter=
2,
3,title
4,REMARKS


In [25]:
df.shape

(5765, 1)

In [26]:
df.isna().sum()

0    0
dtype: int64

In [27]:
df[0].replace("   ", np.nan, inplace=True)
df[0].replace("  ", np.nan, inplace=True)
df[0].replace(" ", np.nan, inplace=True)
df[0].replace("", np.nan, inplace=True)

In [28]:
df.isna().sum()

0    1238
dtype: int64

In [29]:
df.dropna(inplace=True)

In [30]:
df.head()

,0
1,chapter=
3,title
4,REMARKS
5,title
6,ON THE


In [31]:
df.shape

(4527, 1)

In [32]:
en1893_str_markers_split = list(df[0])

In [33]:
en1893_str_markers_split[11]

' 2 I. 31; ii. 259. '

## Reconstruct text without markers

In [1234]:
en_reconstructed = []
en_reconstructed_idx = -1

chapter_counter = -1
chap2idx_start = {}

book_counter = 0

idx2section_name = {}

hit_section_marker = False

lst = []

for section_idx, section in enumerate(en1893_str_markers_split):
    lst.append(hit_section_marker)
    
    if section == "":
        continue
    elif section == " ":
        continue
    elif section == "  ":
        continue
        
    # if section is chapter marker
    elif re.match(chapter_marker, section):
        # increment chapter counter
        chapter_counter += 1
        # reset book counter
        book_counter = 0
        # mark idx of en_reconstructed where chapter starts
        chap2idx_start[chapter_counter] = en_reconstructed_idx
    
    # if section is title marker
    elif re.match(title_marker, section):
        hit_section_marker = True
        # then next section is a title, which will be added to en_reconstructed
        # label as title for titles that don't match instances below (index, book, argument)
        label_nextitem = str(chapteridx2chaptername[chapter_counter]) + "book" + str(book_counter) + "title"
        
        # if title (next item in list) starts with "INDEX"
        if en1893_str_markers_split[section_idx+1].startswith("INDEX."):
            # index is in one section only; rename label
            label_nextitem = "index"

        # if next item in list starts with "BOOK"
        elif en1893_str_markers_split[section_idx+1].startswith("BOOK"):
            # new book is starting
            book_counter += 1
            # rename label
            label_nextitem = str(chapteridx2chaptername[chapter_counter]) + "book" + str(book_counter) + "title"
        
        # if next item in list starts with "ARGUMENT"
        elif en1893_str_markers_split[section_idx+1].startswith("ARGUMENT"):
            # label as commentary
            label_nextitem = str(chapteridx2chaptername[chapter_counter]) + "book" + str(book_counter) + "commentary"
        
        continue
    
    # if section marks the start of a note
    # elif re.match(note_start, section):
    elif section == "note":
        if en1893_str_markers_split[section_idx+1] == "note_end":
            hit_section_marker = False
            continue
        else:
            hit_section_marker = True
            # then next section is a footnote, add to en_reconstructed
            label_nextitem = str(chapteridx2chaptername[chapter_counter]) + "book" + str(book_counter) + "note"
    
    elif re.match(note_end, section):
        # do nothing; just there to separate footnote from following text
        hit_section_marker = False
        continue
    
    elif re.match(para_end, section):
        # do nothing; just there to create separations in sections from raw text
        hit_section_marker = False
        continue
    
    else:
        # all markers have been excluded; append to en_reconstructed
        en_reconstructed.append(section)
        en_reconstructed_idx += 1
        
        if hit_section_marker == True:
            idx2section_name[en_reconstructed_idx] = label_nextitem
            hit_section_marker = False
            
        else:
            idx2section_name[en_reconstructed_idx] = str(chapteridx2chaptername[chapter_counter]) + "book" + str(book_counter)


In [1235]:
# fix errors here, then initialize en1983_sents_tokenized again
for sent_idx, sent in enumerate(en_reconstructed):
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace(".”1", ".” 1")
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace("tongue.2", "tongue. 2")
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace("veneration.3", "veneration. 3")
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace("insane.3", "insane. 3")
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace(".1", ". 1")
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace("opinion.3", "opinion. 3")
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace(".7", ". 7")
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace(".2", ". 2")
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace(".4", ". 4")
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('language;"', 'language; "')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('Lucretius;"', 'Lucretius; "')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace("?1", "? 1")
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace(".5", ". 5")
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace(".6", ". 6")
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace("It;.", "It; .")
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace(":8", ": 8")
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace(".3", ". 3")
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace(":2", ": 2")
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace("?2", "? 2")
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('ἔχειπρός.”Wakefield.', 'ἔχειπρός. ”Wakefield.')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace("Tacitus. 11", "Tacitus.11")
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('Æolia.”What', 'Æolia. ”What')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('concerned).—', 'concerned). —')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('.—', '. —')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace("nix.—Are", "nix. —Are")
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace("improvida.—Find", "improvida. —Find")
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace(";", "; ")
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace("fleshes.”", "fleshes. ”")
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace(".]Ver", ".] Ver")
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace(":so", ": so")
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace("flumina. —“", "flumina.—“")
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace("armis.–Fortè", "armis. –Fortè")
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace("—But", "— But")
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace("—Or", "— Or")
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace("minds.", "minds. ")
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace(':"', ': "')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('first.-The', 'first. -The')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace(':)', ': )')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('body. ——I', 'body.— —I')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('curtains. —“', 'curtains.—“')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('593. —"', '593.—"')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('borne. 3', 'borne.3')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('words.J', 'words. J')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('compose. 1', 'compose.1')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace("systems.”Whewell's", "systems. ”Whewell's")
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('life?3', 'life? 3')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace("? 1  Or,", "?1 Or,")
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('true.]1', 'true.] 1')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('Phil. i. 1. 1.', 'Phil. i. 1.1.')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('up. 1 adhering', 'up.1 adhering')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('6, 6. —With', '6, 6.—With')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('THE YEAR. 3', 'THE YEAR.3')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('elbows?3', 'elbows? 3')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace(':—idem', ': —idem')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('moisture.-Motion', 'moisture. -Motion')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('ENTIRE WHOLE. 1', 'ENTIRE WHOLE.1')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('sui.”Aul.Gell.', 'sui. ”Aul. Gell.')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('itself?3', 'itself? 3')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('sun?4', 'sun? 4')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('day :—', 'day : —')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('every nerve. — ', 'every nerve.—')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('fury !— But', 'fury ! —But')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('flame. —', 'flame.—')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('concealed. —', 'concealed.—')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('rose:-', 'rose: -')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('exhaustion:—such', 'exhaustion: —such')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('beneath.-On', 'beneath. -On')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('bring:-', 'bring: -')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('Corporeal:—hence', 'Corporeal: —hence')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace(':-can', ': -can')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace(':-a', ': -a')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace(':-by', ': -by')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('.And', '. And')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('death.-Since', 'death. -Since')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace(':-some', ': -some')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('ever!"— But', 'ever!" —But')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('peace. —', 'peace.—')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('aid.-No', 'aid. -No')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('man. —', 'man.—')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('?or', '? or')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace(':-it', ': -it')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('ill. —', 'ill.—')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('prevail.-Thus', 'prevail. -Thus')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace(':-for', ': -for')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('well-spring:—creed', 'well-spring: —creed')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace(':different', ': different')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('an:!', 'an: !')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('v. 2. 5', 'v. 2.5')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('v. 1116;', 'v.1116;')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('1. 57,', '1.57,')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace(':voluntary', ': voluntary')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('ib. ;   grow', 'ib. grow')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('in, v. 30', 'in, v.30')    

# Tokenize en1893 with and without section markers

In [1239]:
en1983_sents_tokenized = []
for idx, sent in enumerate(en1983_sents):
    en1983_sents_tokenized.append(sent.split())

In [1240]:
en_reconstructed_tokenized = []
for idx, sent in enumerate(en_reconstructed):
    en_reconstructed_tokenized.append(sent.split())

In [1241]:
num_tokens_sents = 0
for sent in en1983_sents_tokenized:
    num_tokens_sents += len(sent)

num_tokens_chapts = 0
for sent in en_reconstructed_tokenized:
    num_tokens_chapts += len(sent)

In [1242]:
num_tokens_chapts == num_tokens_sents

True

In [1243]:
print(num_tokens_chapts)
print(num_tokens_sents)

202635
202635


## Find data errors in tokenized docs

In [1244]:
en1983_tokens_from_sents = flatten_list(en1983_sents_tokenized)
en1983_tokens_from_chapts = flatten_list(en_reconstructed_tokenized)

In [1245]:
en1983_tokens_from_sents == en1983_tokens_from_chapts

True

In [1246]:
en1983_tokens_from_sents[201703:201709]

['in,', 'v.30', 'Thunder,', 'not', 'supernatural,', 'vi.']

In [1247]:
en1983_tokens_from_chapts[201703:201709]

['in,', 'v.30', 'Thunder,', 'not', 'supernatural,', 'vi.']

In [1248]:
for idx, token in enumerate(en1983_tokens_from_chapts):
    if token != en1983_tokens_from_sents[idx]:
        print(idx)

# Get sent idx to section name dict for en1893

In [1258]:
def build_sent_to_section_dict(lst_tokenized_sents, lst_tokenized_chapts,
                               dict_chapter_2_section):
    """
    Build dict of sentence idx to section name
    """
    sent_idx_2_section_name = {}
    token_counter = 0 # per section/chapter
    current_section_idx = 0
    for idx_sent, sent in enumerate(lst_tokenized_sents):
        token_counter += len(sent)
        current_chapter_length = len(lst_tokenized_chapts[current_section_idx])
        if token_counter < current_chapter_length:
            # add sent to dict
            sent_idx_2_section_name[idx_sent] = dict_chapter_2_section[current_section_idx]
        elif token_counter == current_chapter_length:
            # add sent to dict as part of current section
            sent_idx_2_section_name[idx_sent] = dict_chapter_2_section[current_section_idx]
            # reset token counter and current section idx for next sent iteration
            token_counter = 0
            current_section_idx += 1
        else: # token_counter > current_chapter_length, i.e. we've crossed a section boundary 
            # add sent to current section and next section
            sent_idx_2_section_name[idx_sent] = [
                dict_chapter_2_section[current_section_idx], 
                dict_chapter_2_section[current_section_idx+1]]
            # adjust token counter by only including portion of sent in new section
            token_counter = token_counter - current_chapter_length
            # update current section idx for next sent iteration
            current_section_idx += 1
    return sent_idx_2_section_name

In [1260]:
en1893_sent2section_name = build_sent_to_section_dict(
    en1983_sents_tokenized, en_reconstructed_tokenized, idx2section_name)


In [1261]:
en1893_sent2section_name

{0: 'forewordbook0title',
 1: 'forewordbook0title',
 2: 'forewordbook0title',
 3: 'forewordbook0title',
 4: 'forewordbook0title',
 5: 'forewordbook0title',
 6: 'forewordbook0title',
 7: 'forewordbook0title',
 8: 'forewordbook0title',
 9: 'forewordbook0title',
 10: 'forewordbook0title',
 11: 'forewordbook0title',
 12: 'forewordbook0title',
 13: 'forewordbook0title',
 14: 'forewordbook0title',
 15: 'forewordbook0title',
 16: 'forewordbook0title',
 17: ['forewordbook0title', 'forewordbook0note'],
 18: ['forewordbook0note', 'forewordbook0note'],
 19: ['forewordbook0note', 'forewordbook0note'],
 20: ['forewordbook0note', 'forewordbook0note'],
 21: ['forewordbook0note', 'forewordbook0note'],
 22: ['forewordbook0note', 'forewordbook0'],
 23: 'forewordbook0',
 24: 'forewordbook0',
 25: 'forewordbook0',
 26: 'forewordbook0',
 27: 'forewordbook0',
 28: 'forewordbook0',
 29: 'forewordbook0',
 30: 'forewordbook0',
 31: 'forewordbook0',
 32: 'forewordbook0',
 33: 'forewordbook0',
 34: 'forewordbook

## Write to json

In [1264]:
# path_out = "/home/craig.car/repos/chiron/align_texts_project/data/lucretius/en1893_sent2section_dict.json"
# with open(path_out, 'w') as fp:
#     json.dump(en1893_sent2section_name, fp)